In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import pandas as pd
import torch
from peft import PeftModel
from pathlib import Path

ASSETS_DIR = Path("./assets")
dataset_path = ASSETS_DIR / "fine-tuning-small.csv" 
eval_dataset_path = ASSETS_DIR / "evaluation-small.csv"
eval_data_output_path = ASSETS_DIR / "evaluation-small-output.csv"

# Temp
OUTPUT_DIR =  "./output/deepseek_coder_v2"

# Models
MODEL_DIR = Path("./models")
WEIGHTS_PATH = MODEL_DIR / 'model_weights_ast.pth'
MODEL_PATH= MODEL_DIR / 'model_peft'

In [ ]:

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("JetBrains/deepseek-coder-1.3B-kexer")
model = torch.load(WEIGHTS_PATH, map_location="cuda" if torch.cuda.is_available() else "cpu")
peft_model = PeftModel.from_pretrained(model, MODEL_PATH)
peft_model.to("cuda" if torch.cuda.is_available() else "cpu")

def generate_solution(example_problem: str) -> str:
    instruction = "You are a coding assistant. Given the following coding problem, provide a clear and detailed solution.\n"
    input_text = instruction + example_problem
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(peft_model.device)
    generated_ids = peft_model.generate(
        input_ids,
        max_length=20000,
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id
    )
    solution = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return solution

In [ ]:
print("Generating solutions for each problem...")
data = pd.read_csv(eval_dataset_path)
# Generate 101 solutions for each problem
for i in range(1, 101):
    column_name = f"solution_{i}"
    data[column_name] = data['problem'].apply(generate_solution)
    print(f"Generated column: {column_name}")

# Save the updated dataset with generated solutions
data.to_csv(eval_data_output_path, index=False)

print("Solutions saved to {eval_data_output_path}")